In [62]:
import os
import cv2
import json
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# データセットのディレクトリ
annotations_dir = "dataset/annotations"
images_dir = "dataset/Set_A_02/images"


# 画像とラベルのデータを読み込む
def load_dataset(annotations_dir, images_dir):
    data = []
    annotations_data = []

    # アノテーションファイルの読み込み
    for annotation_file in os.listdir(annotations_dir):
        with open(os.path.join(annotations_dir, annotation_file), "r") as file:
            annotations = json.load(file)

        # 画像ファイルの読み込み
        for k, v in annotations.items():
            image_path = os.path.join(images_dir, v["filename"])
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, (224, 224))  # 画像のリサイズ
                data.append(image)

            annotations_data.append({k : v})

    return np.array(data), annotations_data


data, labels = load_dataset(annotations_dir, images_dir)

In [86]:
from sklearn.preprocessing import LabelEncoder


def encode_labels(labels, attr):
    label_encoder = LabelEncoder()
    encoded_labels = []

    for label in labels:
        for k, v in label.items():
            label = v["regions"][0]["region_attributes"][f'{attr}']
            encoder = label_encoder.fit_transform([label])
            encoded_labels.append(encoder)
    return np.array(encoded_labels)


encode_floor_labels = encode_labels(labels, "floor_label")
encode_class_labels = encode_labels(labels, 'class')
encode_description_labels = encode_labels(labels, 'description')

In [87]:
encode_floor_labels
encode_class_labels
encode_description_labels

array([[0],
       [0]])

In [83]:
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(np.unique(encoded_labels).size, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


In [84]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

from sklearn.model_selection import train_test_split

# データセットをトレーニング用とテスト用に分割
train_data, test_data, train_labels, test_labels = train_test_split(
    data, encoded_labels, test_size=0.2
)

# モデルのトレーニング
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Epoch 1/10


/home/onikarubi/documents/team-shiny-products/floor-plan-ocr/venv/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━

In [85]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Test accuracy: 0.00%
